In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ecephys_analyses.data import paths, channel_groups
from ecephys.signal.sharp_wave_ripples import apply_ripple_filter
from ecephys.sglx_utils import load_timeseries
from ecephys.plot import lfp_explorer
import ecephys.signal.timefrequency as tfr

In [3]:
import pandas as pd
import numpy as np
from ripple_detection.core import get_envelope
import seaborn as sns
import matplotlib.pyplot as plt
import xarray as xr
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime

In [4]:
def parallel_spectrogram_welch(sig, **kwargs):
    freqs, spg_times, spg = tfr.parallel_spectrogram_welch(sig.values, sig.fs, **kwargs)
    spg_times = spg_times + sig.time.values.min()
    return xr.DataArray(
        spg,
        dims=("frequency", "time", "channel"),
        coords={"frequency": freqs, "time": spg_times, "channel": sig.channel.values},
        attrs={'units': f"{sig.units}^2/Hz", 'file_start': sig.fileCreateTime}
    )

In [5]:
def get_bandpower(spg, f_range, chans):
    bandpower = spg.sel(frequency=slice(*f_range)).sum(
        dim="frequency"
    )
    bandpower.attrs["freq_range"] = f_range
    bandpower.attrs['chans'] = chans

    return bandpower

In [6]:
def plot_bandpower_vs_channel_vs_time(da, figsize=(36, 8)):
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(da.T, xticklabels=da.time.values.round(), yticklabels=da.channel.values, cbar=False)
    ax.set(xticks=ax.get_xticks()[::2], yticks=ax.get_yticks()[::4], xlabel='Epoch center time (s)', ylabel='Channel')

In [7]:
def plot_bandpower_vs_channel(da, figsize=(36, 5)):
    fig, ax = plt.subplots(figsize=figsize)
    channel_indices = np.arange(len(da.channel))
    sns.barplot(x=channel_indices, y=da, color='steelblue', ax=ax)
    ax.set(xlabel="Channel", ylabel="Power")
    ax.set_xticks(channel_indices);
    ax.set_xticklabels(da.channel.values, rotation=90);

In [8]:
def create_drift_report(spg, epoch_length, pdf_path):
    
    theta_range = (5, 10)
    theta_power = get_bandpower(spg, theta_range, spg.channel.values)
    epoched_theta_power = theta_power.coarsen(time=epoch_length, boundary='trim').mean()

    ripple_range = (150, 250)
    ripple_power = get_bandpower(spg, ripple_range, spg.channel.values)
    epoched_ripple_power = ripple_power.coarsen(time=epoch_length, boundary='trim').mean()
    
    pdf_path = pdf_path.parent / (pdf_path.stem + f'-{epoch_length}s' + pdf_path.suffix)
    with PdfPages(pdf_path) as pdf:
        plot_bandpower_vs_channel_vs_time(epoched_theta_power)
        plt.title(f'Theta power, {theta_range}, {epoch_length}s epochs')
        pdf.savefig()
        plt.close()

        plot_bandpower_vs_channel_vs_time(epoched_ripple_power)
        plt.title(f'Ripple power, {ripple_range}, {epoch_length}s epochs')
        pdf.savefig()
        plt.close()

        assert len(epoched_theta_power) == len(epoched_ripple_power)
        for epoch in range(len(epoched_theta_power)):
            plot_bandpower_vs_channel(epoched_theta_power[epoch])
            plt.title(f'Theta power, epoch: {epoch}, time: {epoched_theta_power[epoch].time.values}s')
            pdf.savefig()
            plt.close()

            plot_bandpower_vs_channel(epoched_ripple_power[epoch])
            plt.title(f'Ripple power, epoch: {epoch}, time: {epoched_theta_power[epoch].time.values}s')
            pdf.savefig()
            plt.close()

        d = pdf.infodict()
        d['Title'] = f'{pdf_path.stem}'
        d['Author'] = 'Graham Findlay'
        d['Subject'] = 'Power profiles for tracking CA1 drift'
        d['CreationDate'] = datetime.now()

In [9]:
def create_drift_reports(bin_path, pdf_path, chans):
    sig = load_timeseries(bin_path, chans)
    spg = parallel_spectrogram_welch(sig)
    
    create_drift_report(spg, 600, pdf_path)
    create_drift_report(spg, 300, pdf_path)
    create_drift_report(spg, 60, pdf_path)

In [32]:
subject = "Doppio"

In [39]:
drift_tracking_chans = channel_groups.drift_tracking[subject]

bin_paths = paths.get_sglx_style_datapaths(subject=subject, condition="all-dark-periods", ext="lf.bin")
pdf_paths = paths.get_sglx_style_datapaths(subject=subject, condition="all-dark-periods", ext="drift.pdf")

for bin_path, pdf_path in zip(bin_paths, pdf_paths):
    create_drift_reports(bin_path, pdf_path, drift_tracking_chans)
    current_time = datetime.now().strftime("%H:%M:%S")
    print(f"{current_time}: Finished {str(bin_path)}")

nChan: 385, nFileSamp: 18000019
20:51:22: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/3-17-2020/3-17-2020_g0/3-17-2020_g0_imec0/3-17-2020_g0_t6.imec0.lf.bin
nChan: 385, nFileSamp: 18000019
21:01:35: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/3-17-2020/3-17-2020_g0/3-17-2020_g0_imec0/3-17-2020_g0_t7.imec0.lf.bin
nChan: 385, nFileSamp: 18000019
21:11:46: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/3-17-2020/3-17-2020_g0/3-17-2020_g0_imec0/3-17-2020_g0_t8.imec0.lf.bin
nChan: 385, nFileSamp: 18000019
21:22:28: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/3-17-2020/3-17-2020_g0/3-17-2020_g0_imec0/3-17-2020_g0_t9.imec0.lf.bin
nChan: 385, nFileSamp: 18000019
21:33:32: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/3-17-2020/3-17-2020_g0/3-17-2020_g0_imec0/3-17-2020_g0_t10.imec0.lf.bin
nChan: 385, nFileSamp: 17715805
21:44:44: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/3-17-202

KeyError: 'nSavedChans'